In [ ]:
### imports

# external modules
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
import DataLoader
import PlotStyleParser
importlib.reload(DataLoader)
importlib.reload(PlotStyleParser)
sys.path.append('../src/classifiers')
import LandauClassifier
importlib.reload(LandauClassifier)

In [ ]:
### load test histograms
    
histname = 'chargeInner_PXLayer_2'
dffile = '../data/DF2017_{}.csv'.format(histname)
dl = DataLoader.DataLoader()
histdf = dl.get_dataframe_from_file(dffile)
histdf = dfu.select_dcson(histdf)
histdf = dfu.select_highstat(histdf)
print(len(histdf))
xmin = histdf.at[0, 'Xmin']
xmax = histdf.at[0, 'Xmax']

(hists_train) = hu.preparedatafromdf(histdf, donormalize=True, doplot=False)
print(hists_train.shape)

In [ ]:
### define a LandauClassifier

importlib.reload(LandauClassifier)
classifier = LandauClassifier.LandauClassifier( dogauss=False )

In [ ]:
### do some example reconstructions

inds = range(50896,50897)

hists = np.zeros((len(inds),hists_train.shape[1]))
recos = np.zeros((len(inds),hists_train.shape[1]))

starttime = time.time()
for i, ind in enumerate(inds):
    hists[i,:] = hists_train[ind,:]
    recos[i,:] = classifier.reconstruct(hists[i:i+1])
endtime = time.time()
print('reconstruction finished in {} seconds'.format(endtime-starttime))

for i in range(len(hists)):
    hist = hists[i]
    reco = recos[i]
    
    # with initial parameters
    landaumax = np.argmax(hist)
    landauwidth = landaumax/6.
    norm = np.sum(hist)/4.
    gausswidth = landauwidth
    xdata = np.arange(0, len(hist))
    landau = classifier.fitfunc( xdata, landaumax, landauwidth, norm )
    pu.plot_hists( [hist,landau], 
                    colorlist=['black','red'], 
                    labellist=['orig','landau'] )
    
    # do fit
    pu.plot_hists( [hist,reco], 
                    colorlist=['black','red'], 
                    labellist=['orig','reco'] )

In [ ]:
xdata = np.arange(0, 100)
landaumax = 20
landauwidth = 5
norm = 1
y = np.divide(xdata-landaumax,landauwidth)
landau = norm/np.sqrt(2.*np.pi)*np.exp( -(y+np.exp(-y))/2. )
plt.figure()
plt.plot(xdata, landau)

gausswidth = 0.1
xgdata = np.linspace(-5*gausswidth,5*gausswidth,num=100)
gauss = np.exp( -0.5*np.power(xgdata/gausswidth,2) )/(np.sqrt(2*np.pi)*gausswidth)
plt.figure()
plt.plot(xgdata, gauss)

conv = np.convolve(landau, gauss, mode='same')
plt.figure()
plt.plot(xdata, conv)

In [ ]:
### make plots for officially illustrating the method

# load aesthetics
plotstyleparser = PlotStyleParser.PlotStyleParser('../internal/pixel_case_study/plotstyle.json')
titledict = plotstyleparser.get_title()
xaxtitledict = plotstyleparser.get_xaxtitle()
yaxtitledict = plotstyleparser.get_yaxtitle()
extratextdict = plotstyleparser.get_extratext()

# other settings
xmin = 0
xmax = 1

# define runs and lumisections 
runsls = [(305842,861), (305842,862)]

for run,ls in runsls:
    
    # get histogram
    dfr = dfu.select_runs(histdf,[run])
    dfr = dfu.select_ls(dfr,[ls])
    hists = hu.preparedatafromdf(dfr, donormalize=True, cropslices=[slice(1,-1)])
    hist = hists[0]
    
    # do reconstruction
    fitted, params = classifier.fit(hist)
    params[0] = params[0]*xmax/len(hist)
    params[1] = params[1]*xmax/len(hist)
    mse = aeu.mseTopNRaw( np.expand_dims(hist,axis=0), np.expand_dims(fitted,axis=0) )[0]
    
    # make figure
    fig,ax = pu.plot_hists( [hist,fitted], colorlist=['b','r'],
                                title = titledict[histname], titlesize=15,
                                xlims=(xmin,xmax),
                                xaxtitle = xaxtitledict[histname].replace('electrons','normalized'), xaxtitlesize=17,
                                yaxtitle = yaxtitledict[histname]+' (normalized)', yaxtitlesize=17,
                                ymaxfactor = 1,
                                legendsize = 14, ticksize = 14 )
    pu.add_cms_label( ax, pos=(0.01,1.01), extratext='Preliminary', fontsize=16 )
    pu.add_text( ax, '2017 (13 TeV)', (0.75,1.01), fontsize=14 )
    pu.add_text( ax, extratextdict[histname], (0.97,0.9), fontsize=15, horizontalalignment='right' )
    pu.add_text( ax, 'Run {}, LS {}'.format(run, ls), (0.97,0.8), fontsize=15, color='blue', horizontalalignment='right' )
    pu.add_text( ax, 'Landau fit'.format(run, ls), (0.97,0.7), fontsize=15, color='red', horizontalalignment='right' )
    pu.add_text( ax, 'center: {:.3f}'.format(params[0]), (0.97,0.6), fontsize=15, color='red', horizontalalignment='right' )
    pu.add_text( ax, 'width: {:.3f}'.format(params[1]), (0.97,0.52), fontsize=15, color='red', horizontalalignment='right' )
    pu.add_text( ax, 'norm: {:.3f}'.format(params[2]), (0.97,0.44), fontsize=15, color='red', horizontalalignment='right' )
    pu.add_text( ax, 'MSE: {:.3e}'.format(mse), (0.97,0.36), fontsize=15, color='red', horizontalalignment='right' )
    plt.show()